In [11]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import reproject 
import concurrent.futures
from astropy.io import fits
import os
from tqdm.auto import tqdm 
import gc 

import sys
sys.path.append('../')
from modules import dendro_cutouts, dendro_misc, dendro_dendro

In [12]:
# Replace with your file dirs here... 
# 
# dir_hst = '../../../data/hst_cropped/'
# hst01_file = '%s/ngc628_uvis_f275w_exp_drc_sci.fits' %dir_hst
# hst02_file = '%s/ngc628_uvis_f336w_exp_drc_sci.fits' %dir_hst
# hst03_file = '%s/ngc628_acs_f435w_exp_drc_sci.fits' %dir_hst
# hst04_file = '%s/ngc628_uvis_f555w_exp_drc_sci.fits' %dir_hst
# hst05_file = '%s/ngc628_acs_f814w_exp_drc_sci.fits' %dir_hst

dir_hst = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/ngc628c/hst_contsub/'
hst06_file = '%s/ngc628_halpha_raw.fits' %dir_hst
hst07_file = '%s/ngc628c_hst_ha_sic.fits' %dir_hst

# 
musha_file = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/ngc628c/muse/NGC0628-0.92asec_MAPS.fits'
# almaco_file = '../../../data/alma/ngc0628_12m+7m+tp_co21_broad_tpeak.fits'
# almaco1_file = '../../../data/alma/ngc0628_12m+7m+tp_co21_broad_mom0.fits'
# vlahi_file = '../../../data/vla/NGC_628_NA_MOM0_THINGS.fits'

# # 
# dir_hst = '../../../data/jwst_updated_v0p4p0/'
# jwst01_file = '%s/ngc0628_nircam_lv3_f200w_i2d_align.fits' %dir_hst
# jwst02_file = '%s/ngc0628_nircam_lv3_f300m_i2d_align.fits' %dir_hst
# jwst03_file = '%s/ngc0628_nircam_lv3_f335m_i2d_align.fits' %dir_hst
# jwst04_file = '%s/ngc0628_nircam_lv3_f360m_i2d_align.fits' %dir_hst

# dir_hst = '../../../data/jwst_updated_v0p5_miri/'
# jwst05_file = '%s/ngc0628_miri_f770w_anchored.fits' %dir_hst
# jwst06_file = '%s/ngc0628_miri_f1000w_anchored.fits' %dir_hst
# jwst07_file = '%s/ngc0628_miri_f1130w_anchored.fits' %dir_hst
# jwst08_file = '%s/ngc0628_miri_f2100w_anchored.fits' %dir_hst


# dir_hst = '../../../data/jwst_feast/'
# jwst09_file = '%s/jw01783-o004_t008_nircam_clear-f115w_i2d_J2000.fits' %dir_hst
# jwst10_file = '%s/jw01783-o004_t008_nircam_clear-f150w_i2d_J2000.fits' %dir_hst
# jwst11_file = '%s/jw01783-o004_t008_nircam_clear-f187n_i2d_J2000.fits' %dir_hst
# jwst12_file = '%s/jw01783-o004_t008_nircam_clear-f187n_ONLY_J2000.fits' %dir_hst
# jwst13_file = '%s/jw01783-o004_t008_nircam_clear-f277w_i2d_J2000.fits' %dir_hst
# jwst14_file = '%s/jw01783-o004_t008_nircam_clear-f444w_i2d_J2000.fits' %dir_hst
# jwst15_file = '%s/jw01783-o004_t008_nircam_f405_ONLY_J2000.fits' %dir_hst
# jwst16_file = '%s/jw01783-o004_t008_nircam_f405n-f444w_i2d_J2000.fits' %dir_hst

# asto_file = '../../../data/astrosat/NGC0628_FUV_ave.fits'
# spit_file = '../../../data/spitzer/NGC_0628_IRAC_I4_bms2014.fits'

# Catalouges
muscat_file = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/ngc628c/muse/NGC0628_nebmask.fits' 
 
dir_asscat = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/ngc0628/data/cluster_catalogs/ngc628-vselect/'
asscat1_file = '%s/ws16pc/PHANGS_IR3_hst_wfc3_ngc628_v1p3_multi_assoc-vselect-ws16pc-idmask.fits' %dir_asscat #HST association catalogue
asscat2_file = '%s/ws32pc/PHANGS_IR3_hst_wfc3_ngc628_v1p3_multi_assoc-vselect-ws32pc-idmask.fits' %dir_asscat #HST association catalogue
asscat3_file = '%s/ws64pc/PHANGS_IR3_hst_wfc3_ngc628_v1p3_multi_assoc-vselect-ws64pc-idmask.fits' %dir_asscat #HST association catalogue

In [13]:
# # loading files
# # changing everything into log scale
# hst01_hdu = fits.open(hst01_file)[0]
# hst02_hdu = fits.open(hst02_file)[0]
# hst03_hdu = fits.open(hst03_file)[0]
# hst04_hdu = fits.open(hst04_file)[0]
# hst05_hdu = fits.open(hst05_file)[0]
# hst06_hdu = fits.open(hst06_file)[0]
hst07_hdu = fits.open(hst07_file)[0]

#  
musha_hdu = fits.open(musha_file)['HA6562_FLUX']
musha_hdu.data[np.isnan(musha_hdu.data)] = -100

# almaco_hdu = fits.open(almaco_file)[0]
# almaco1_hdu = fits.open(almaco1_file)[0]

# vlahi_hdu = fits.open(vlahi_file)[0]
# vlahi_hdu.data = np.squeeze(vlahi_hdu.data)
# del vlahi_hdu.header['*3*']
# del vlahi_hdu.header['*4*']

#
# jwst01_hdu = fits.open(jwst01_file)[1]
# jwst02_hdu = fits.open(jwst02_file)[1]
# jwst03_hdu = fits.open(jwst03_file)[1]
# jwst04_hdu = fits.open(jwst04_file)[1]
# jwst05_hdu = fits.open(jwst05_file)[0]
# jwst06_hdu = fits.open(jwst06_file)[0]
# jwst07_hdu = fits.open(jwst07_file)[0]
# jwst08_hdu = fits.open(jwst08_file)[0]

# jwst01_hdu.data[jwst01_hdu.data == 0] = np.nan
# jwst02_hdu.data[jwst02_hdu.data == 0] = np.nan
# jwst03_hdu.data[jwst03_hdu.data == 0] = np.nan
# jwst04_hdu.data[jwst04_hdu.data == 0] = np.nan
# jwst05_hdu.data[jwst05_hdu.data == 0] = np.nan
# jwst06_hdu.data[jwst06_hdu.data == 0] = np.nan
# jwst07_hdu.data[jwst07_hdu.data == 0] = np.nan
# jwst08_hdu.data[jwst08_hdu.data == 0] = np.nan

# jwst09_hdu = fits.open(jwst09_file)[0]
# jwst10_hdu = fits.open(jwst10_file)[0]
# jwst11_hdu = fits.open(jwst11_file)[0]
# jwst12_hdu = fits.open(jwst12_file)[0]
# jwst13_hdu = fits.open(jwst13_file)[0]
# jwst14_hdu = fits.open(jwst14_file)[0]
# jwst15_hdu = fits.open(jwst15_file)[0]
# jwst16_hdu = fits.open(jwst16_file)[0]

# jwst09_hdu.data[jwst09_hdu.data == 0] = np.nan
# jwst10_hdu.data[jwst10_hdu.data == 0] = np.nan
# jwst11_hdu.data[jwst11_hdu.data == 0] = np.nan
# jwst12_hdu.data[jwst12_hdu.data == 0] = np.nan
# jwst13_hdu.data[jwst13_hdu.data == 0] = np.nan
# jwst14_hdu.data[jwst14_hdu.data == 0] = np.nan
# jwst15_hdu.data[jwst15_hdu.data == 0] = np.nan
# jwst16_hdu.data[jwst16_hdu.data == 0] = np.nan

#
# asto_hdu = fits.open(asto_file)[0]
# spit_hdu = fits.open(spit_file)[0]

# data, _ = reproject.reproject_interp(spit_hdu, musha_hdu.header)
# spit_hdu = fits.PrimaryHDU(data, musha_hdu.header)

# loading catalouges
muscat_hdu = fits.open(muscat_file)[0]
muscat_hdu.data = np.array(muscat_hdu.data, dtype=float)
muscat_hdu.data[muscat_hdu.data==-1] = np.nan

if os.path.isfile(asscat1_file.replace('.fits', 'regrid.fits')):
    
    print('[INFO] [load_fits] Importing regrid asscat_files...')
    asscat1_hdu = fits.open(asscat1_file.replace('.fits', 'regrid.fits'))[0]
    asscat2_hdu = fits.open(asscat2_file.replace('.fits', 'regrid.fits'))[0]
    asscat3_hdu = fits.open(asscat3_file.replace('.fits', 'regrid.fits'))[0]

else: 
    
    print('[INFO] [load_fits] No regrid asscat_files, making them now - could take a min...')
    asscat1_hdu = fits.open(asscat1_file)[0]
    asscat2_hdu = fits.open(asscat2_file)[0]
    asscat3_hdu = fits.open(asscat3_file)[0]

    asscat1_hdu = fits.PrimaryHDU(reproject.reproject_interp(asscat1_hdu, hst07_hdu.header, return_footprint=False, order='nearest-neighbor', parallel=True), hst07_hdu.header)
    asscat2_hdu = fits.PrimaryHDU(reproject.reproject_interp(asscat2_hdu, hst07_hdu.header, return_footprint=False, order='nearest-neighbor', parallel=True), hst07_hdu.header)
    asscat3_hdu = fits.PrimaryHDU(reproject.reproject_interp(asscat3_hdu, hst07_hdu.header, return_footprint=False, order='nearest-neighbor', parallel=True), hst07_hdu.header)

    asscat1_hdu.data[asscat1_hdu.data==0] = np.nan
    asscat2_hdu.data[asscat2_hdu.data==0] = np.nan
    asscat3_hdu.data[asscat3_hdu.data==0] = np.nan

    asscat1_hdu.writeto(asscat1_file.replace('.fits', 'regrid.fits'))
    asscat2_hdu.writeto(asscat2_file.replace('.fits', 'regrid.fits'))
    asscat3_hdu.writeto(asscat3_file.replace('.fits', 'regrid.fits'))

[INFO] [load_fits] Importing regrid asscat_files...


In [14]:
# hdus = [hst01_hdu, hst02_hdu, hst03_hdu, hst04_hdu, hst05_hdu, hst06_hdu, hst07_hdu, musha_hdu,
#         almaco_hdu, almaco1_hdu, vlahi_hdu, jwst01_hdu, jwst02_hdu, jwst03_hdu, jwst04_hdu,
#         jwst05_hdu, jwst06_hdu, jwst07_hdu, jwst08_hdu, jwst09_hdu, jwst10_hdu, jwst11_hdu,
#         jwst12_hdu, jwst13_hdu, jwst14_hdu, jwst15_hdu, jwst16_hdu, asto_hdu, spit_hdu,
#         muscat_hdu, asscat1_hdu, asscat2_hdu, asscat3_hdu]

hdus = [hst07_hdu, musha_hdu, muscat_hdu, asscat1_hdu, asscat2_hdu, asscat3_hdu]

# Define the conversion function
def convert_to_float32(hdu):
    hdu.data = hdu.data.astype('float32')

# Create a ThreadPoolExecutor with the number of workers
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit conversion tasks to the executor
    futures = [executor.submit(convert_to_float32, hdu) for hdu in hdus]

    # Wait for all tasks to complete and track the progress
    with tqdm(total=len(futures), desc="Converting to float32") as pbar:
        for future in concurrent.futures.as_completed(futures):
            # Update the progress bar for each completed task
            pbar.update(1)

Converting to float32:   0%|          | 0/6 [00:00<?, ?it/s]

In [15]:
galaxy = 'ngc0628'
regions_file = './analysis/catalogue/sample_muse.reg'

if os.path.exists(regions_file):
    regions = dendro_cutouts.get_regions(regions_file)
else: 
    muscat_table = dendro_dendro.get_MuseProps(galaxy)
    dendro_cutouts.get_ds9regions_sq_decor(muscat_table, outputfile=regions_file)

[INFO] [get_regions] Opening regions file (this may take a min)... 
[INFO] [get_regions] Getting info for 2369 regions...


In [16]:
# hdus = dendro_cutouts.get_croppeddata_all(hst07_hdu, regions)
# dendro_misc.save_pickle(hdus, './analysis/catalogue/hst07_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = dendro_cutouts.get_croppeddata_all(musha_hdu, regions)
# dendro_misc.save_pickle(hdus, './analysis/catalogue/musha_hdus.pickel')
# del hdus
# _ = gc.collect()

hdus = dendro_cutouts.get_croppeddata_all(muscat_hdu, regions)
dendro_misc.save_pickle(hdus, './analysis/catalogue/muscat_hdus.pickel')
del hdus

# hdus = dendro_cutouts.get_croppeddata_all(asscat1_hdu, regions)
# dendro_misc.save_pickle(hdus, './analysis/catalogue/asscat1_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = dendro_cutouts.get_croppeddata_all(asscat2_hdu, regions)
# dendro_misc.save_pickle(hdus, './analysis/catalogue/asscat2_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = dendro_cutouts.get_croppeddata_all(asscat3_hdu, regions)
# dendro_misc.save_pickle(hdus, './analysis/catalogue/asscat3_hdus.pickel') 
# del hdus
# _ = gc.collect()

Cropping regions:   0%|          | 0/2369 [00:00<?, ?it/s]

[INFO] [save_pickle] Saved to ./analysis/catalogue/muscat_hdus.pickel


In [17]:
hst07_hdus = dendro_misc.load_pickle('./analysis/catalogue/hst07_hdus.pickel')
musha_hdus = dendro_misc.load_pickle('./analysis/catalogue/musha_hdus.pickel')
muscat_hdus = dendro_misc.load_pickle('./analysis/catalogue/muscat_hdus.pickel')
asscat1_hdus = dendro_misc.load_pickle('./analysis/catalogue/asscat1_hdus.pickel')
asscat2_hdus = dendro_misc.load_pickle('./analysis/catalogue/asscat2_hdus.pickel')
asscat3_hdus = dendro_misc.load_pickle('./analysis/catalogue/asscat3_hdus.pickel')

hdus = {'hst07_hdus' : hst07_hdus,
        'musha_hdus' : musha_hdus,
        'muscat_hdus' : muscat_hdus,
        'asscat1_hdus' : asscat1_hdus,
        'asscat2_hdus' : asscat2_hdus,
        'asscat3_hdus' : asscat3_hdus}

dendro_misc.save_pickle(hdus, './analysis/catalogue/hdus_sample_sci.pickel') 

[INFO] [load_pickle] Load ./analysis/catalogue/hst07_hdus.pickel
[INFO] [load_pickle] Load ./analysis/catalogue/musha_hdus.pickel
[INFO] [load_pickle] Load ./analysis/catalogue/muscat_hdus.pickel
[INFO] [load_pickle] Load ./analysis/catalogue/asscat1_hdus.pickel
[INFO] [load_pickle] Load ./analysis/catalogue/asscat2_hdus.pickel
[INFO] [load_pickle] Load ./analysis/catalogue/asscat3_hdus.pickel
[INFO] [save_pickle] Saved to ./analysis/catalogue/hdus_sample_sci.pickel
